# 1 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# 핑크색 warning 안내문 방지 
import warnings
warnings.filterwarnings('ignore')

# 2 한글폰트 사용을 위한 세팅

In [3]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(Colab)
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family

plt.rc("font", family=get_font_family())
plt.rc("axes", unicode_minus=False)

# 3 데이터 셋 로드 

In [4]:
import glob 

In [5]:
path = glob.glob('data/*.csv')
path

['data\\animal_raw.csv',
 'data\\dirty_data.csv',
 'data\\관광지.csv',
 'data\\반려견정보.csv']

In [6]:
df = pd.read_csv(path[1], encoding = 'utf-8')
df

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상
0,골든 리트리버,13,0.5,27.0,M,N,갈색
1,골든 리트리버,10,0.5,25.0,F,N,갈색
2,골든 리트리버,9,0.5,40.0,M,N,갈색
3,골든 리트리버,8,0.5,28.0,F,N,갈색
4,골든 리트리버,8,0.5,33.0,M,N,갈색
...,...,...,...,...,...,...,...
7821,핏불테리어,2,1.5,25.0,F,U,검은색
7822,화이트테리어,13,0.5,7.3,F,Y,흰
7823,화이트테리어,5,1.5,5.0,F,U,흰색
7824,휘펫,2,1.0,14.0,M,N,베이지


In [7]:
df_1 = df.copy()

# 4 데이터 살펴보기 

In [8]:
df_1.head()

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상
0,골든 리트리버,13,0.5,27.0,M,N,갈색
1,골든 리트리버,10,0.5,25.0,F,N,갈색
2,골든 리트리버,9,0.5,40.0,M,N,갈색
3,골든 리트리버,8,0.5,28.0,F,N,갈색
4,골든 리트리버,8,0.5,33.0,M,N,갈색


In [9]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7826 entries, 0 to 7825
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   품종        7826 non-null   object 
 1   나이        7826 non-null   int64  
 2   산책시간(시간)  7826 non-null   float64
 3   체중        7826 non-null   float64
 4   성별        7826 non-null   object 
 5   중성화여부     7826 non-null   object 
 6   색상        7826 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 428.1+ KB


In [10]:
df_1.describe()

,나이,산책시간(시간),체중
count,7826.000000,7826.000000,7826.000000
mean,2.099540,0.897304,7.794053
std,2.647694,0.168873,6.967980
min,0.000000,0.500000,0.040000
25%,0.000000,0.800000,3.000000
50%,1.000000,1.000000,5.000000
75%,3.000000,1.000000,11.000000
max,18.000000,1.500000,60.000000


In [11]:
#결측치 확인
df_1.isna().sum()

품종          0
나이          0
산책시간(시간)    0
체중          0
성별          0
중성화여부       0
색상          0
dtype: int64

# 5 데이터 클렌징 

## 5-1 연산을 통한 새로운 컬럼 추가 

In [12]:
#음수량 컬럼 새로 추가 (1kg 당 65ml)
df_1['음수량'] = df_1['체중'] * 65

In [13]:
#성별 컬럼의 데이터 라벨링 해주기 

## 5-2 성별 컬럼 레이블 인코딩 (계층형 아닐땐 이걸로)
- 카테고리 피쳐를 숫자형 값으로 변환해주자

In [14]:
df_1['성별'].unique()

array(['M', 'F', 'Q'], dtype=object)

In [15]:
print(df_1[df_1['성별'].astype(str).str.contains('M')]['성별'].count())
print(df_1[df_1['성별'].astype(str).str.contains('F')]['성별'].count())
print(df_1[df_1['성별'].astype(str).str.contains('Q')]['성별'].count())

3980
3766
80


In [16]:
# 레이블 인코딩에 필요한 라이브러리 로드
from sklearn.preprocessing import LabelEncoder

In [17]:
#성별 컬럼 은 레이블 인코딩으로 변환을 진행해주자 
# LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행
le = LabelEncoder()
le.fit(df_1['성별'])
le.transform(df_1['성별'])
df_1['성별_라벨링'] = le.transform(df_1['성별'])

In [18]:
# 잘 변환 되었는지 확인해주기 
df_1['성별_라벨링'].unique()

array([1, 0, 2])

In [19]:
# 1 : M / 0 : F / 2 : Q(확인불가능)
df_1['성별_라벨링']

0       1
1       0
2       1
3       0
4       1
       ..
7821    0
7822    0
7823    0
7824    1
7825    0
Name: 성별_라벨링, Length: 7826, dtype: int32

In [20]:
#성별을 알 수 없는 애들이 80마리나 ㅠㅠ 전체의 1 퍼센트 ... 이상치라고 생각하고 버리고 갈까 ..? 
df_1['성별_라벨링'].astype(str).str.contains('2').value_counts()

False    7746
True       80
Name: 성별_라벨링, dtype: int64

In [21]:
#성별을 알 수 없는 아이들은 분석에서 제외하고 df_valid 라는 새로운 변수로 선언해준다
df_valid = df_1[df_1['성별_라벨링'].astype(str).str.contains('0') | df_1['성별_라벨링'].astype(str).str.contains('1')]
df_valid.head()

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상,음수량,성별_라벨링
0,골든 리트리버,13,0.5,27.0,M,N,갈색,1755.0,1
1,골든 리트리버,10,0.5,25.0,F,N,갈색,1625.0,0
2,골든 리트리버,9,0.5,40.0,M,N,갈색,2600.0,1
3,골든 리트리버,8,0.5,28.0,F,N,갈색,1820.0,0
4,골든 리트리버,8,0.5,33.0,M,N,갈색,2145.0,1


## 5-3 중성화여부 컬럼 레이블 인코딩 (계층형 아닐땐 이걸로)
- 카테고리 피쳐를 숫자형 값으로 변환해주자

In [22]:
df_valid['중성화여부'].unique()

array(['N', 'Y', 'U'], dtype=object)

In [23]:
print(df_valid[df_valid['중성화여부'].astype(str).str.contains('N')]['중성화여부'].count())
print(df_valid[df_valid['중성화여부'].astype(str).str.contains('Y')]['중성화여부'].count())
print(df_valid[df_valid['중성화여부'].astype(str).str.contains('U')]['중성화여부'].count())

5176
498
2072


In [24]:
#중성화여부 컬럼 은 레이블 인코딩으로 변환을 진행해주자 
# LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행
le = LabelEncoder()
le.fit(df_valid['중성화여부'])
le.transform(df_valid['중성화여부'])
df_valid['중성화_라벨링'] = le.transform(df_valid['중성화여부'])

In [25]:
# 잘 변환 되었는지 확인해주기 
df_valid['중성화_라벨링'].unique()

array([0, 2, 1])

In [26]:
# 0 : N / 1 : U / 2 : Y
df_valid['중성화_라벨링']

0       0
1       0
2       0
3       0
4       0
       ..
7821    1
7822    2
7823    1
7824    0
7825    1
Name: 중성화_라벨링, Length: 7746, dtype: int32

In [27]:
#중성화여부를 알 수 없는 애들이 2072마리나 ㅠㅠ 전체의 30 퍼센트 ... 이상치라고 생각하고 버리고 갈까 ..? 
df_valid['중성화_라벨링'].astype(str).str.contains('1').value_counts()

False    5674
True     2072
Name: 중성화_라벨링, dtype: int64

In [28]:
#중성화여부를 알 수 없는 아이들은 분석에서 제외하고 pets 라는 새로운 변수로 선언해준다
pets = df_valid[df_valid['중성화_라벨링'].astype(str).str.contains('0') | df_valid['중성화_라벨링'].astype(str).str.contains('2')]
pets.head()

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상,음수량,성별_라벨링,중성화_라벨링
0,골든 리트리버,13,0.5,27.0,M,N,갈색,1755.0,1,0
1,골든 리트리버,10,0.5,25.0,F,N,갈색,1625.0,0,0
2,골든 리트리버,9,0.5,40.0,M,N,갈색,2600.0,1,0
3,골든 리트리버,8,0.5,28.0,F,N,갈색,1820.0,0,0
4,골든 리트리버,8,0.5,33.0,M,N,갈색,2145.0,1,0


In [29]:
pets['중성화_라벨링'].unique()

array([0, 2])

In [30]:
#0과 2 보다는 0과 1이 나으니 변경해주자
pets['중성화_라벨링'] = pets['중성화_라벨링'].astype(str).str.replace('2','1')

In [31]:
#2가 1로 잘 변경 되었는지 확인 
pets['중성화_라벨링'].unique()

array(['0', '1'], dtype=object)

In [32]:
pets.shape

(5674, 10)

## 5-4 체중에 따른 크기 컬럼 레이블인코딩
- 카테고리 피쳐를 숫자형 값으로 변환해주자

### 삽질

In [33]:
pets['체중'][1]

25.0

In [34]:
pets['체중'].dtypes

dtype('float64')

In [35]:
## FOR 문으로 돌리려 했는데 실패함ㅠ 

In [36]:
# 계속 오류 뜨니까 걍 원래 정제된 컬럼에서 갖고 와야겠다 ㅠ 
df = pd.read_csv(path[3], encoding = 'utf-8')
df

,품종,크기(체중에따른),크기(라벨링),나이,산책시간(시간),체중,적정음수량,성별,중성화여부,색상
0,골든 리트리버,대,2,13,0.5,27.0,1755.0,M,N,갈색
1,골든 리트리버,대,2,10,0.5,25.0,1625.0,F,N,갈색
2,골든 리트리버,대,2,9,0.5,40.0,2600.0,M,N,갈색
3,골든 리트리버,대,2,8,0.5,28.0,1820.0,F,N,갈색
4,골든 리트리버,대,2,8,0.5,33.0,2145.0,M,N,갈색
...,...,...,...,...,...,...,...,...,...,...
7821,핏불테리어,대,2,2,1.5,25.0,1625.0,F,U,검은색
7822,화이트테리어,소,0,13,0.5,7.3,474.5,F,Y,흰
7823,화이트테리어,소,0,5,1.5,5.0,325.0,F,U,흰색
7824,휘펫,중,1,2,1.0,14.0,910.0,M,N,베이지


In [37]:
print(df[df['중성화여부'].astype(str).str.contains('Q')]['중성화여부'].count())
print(df[df['중성화여부'].astype(str).str.contains('Y')]['중성화여부'].count())
print(df[df['중성화여부'].astype(str).str.contains('N')]['중성화여부'].count())

0
501
5238


In [38]:
# 중성화여부, 성별 판단 안되는애들 제외하기 
df = df[df['성별'].astype(str).str.contains('M') | df['성별'].astype(str).str.contains('F')]
df = df[df['중성화여부'].astype(str).str.contains('Y') | df['중성화여부'].astype(str).str.contains('N')]

In [39]:
df.shape

(5674, 10)

In [40]:
#체중을 기준으로 대/중/소 분류된거 가져와서 pets 데이터프레임에 붙이기 
pets['크기'] = df['크기(체중에따른)']

In [41]:
#잘 나왔는지 확인 
pets

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상,음수량,성별_라벨링,중성화_라벨링,크기
0,골든 리트리버,13,0.5,27.0,M,N,갈색,1755.0,1,0,대
1,골든 리트리버,10,0.5,25.0,F,N,갈색,1625.0,0,0,대
2,골든 리트리버,9,0.5,40.0,M,N,갈색,2600.0,1,0,대
3,골든 리트리버,8,0.5,28.0,F,N,갈색,1820.0,0,0,대
4,골든 리트리버,8,0.5,33.0,M,N,갈색,2145.0,1,0,대
...,...,...,...,...,...,...,...,...,...,...,...
7818,프렌치불독,1,0.8,4.0,M,N,흑,260.0,1,0,소
7819,프렌치불독,0,0.5,3.5,M,N,검정,227.5,1,0,소
7820,프렌치불독,0,0.5,5.2,M,N,"백색, 갈색, 흑색",338.0,1,0,소
7822,화이트테리어,13,0.5,7.3,F,Y,흰,474.5,0,1,소


## 원 핫 인코딩 준비 

In [42]:
pets.head()

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상,음수량,성별_라벨링,중성화_라벨링,크기
0,골든 리트리버,13,0.5,27.0,M,N,갈색,1755.0,1,0,대
1,골든 리트리버,10,0.5,25.0,F,N,갈색,1625.0,0,0,대
2,골든 리트리버,9,0.5,40.0,M,N,갈색,2600.0,1,0,대
3,골든 리트리버,8,0.5,28.0,F,N,갈색,1820.0,0,0,대
4,골든 리트리버,8,0.5,33.0,M,N,갈색,2145.0,1,0,대


In [43]:
pets['크기'].unique()

array(['대', '중', '소'], dtype=object)

In [44]:
print(pets[pets['크기'].astype(str).str.contains('소')]['크기'].count())
print(pets[pets['크기'].astype(str).str.contains('중')]['크기'].count())
print(pets[pets['크기'].astype(str).str.contains('대')]['크기'].count())

4124
1134
416


In [45]:
#크기 컬럼 은 레이블 인코딩으로 변환을 진행해주자 
# LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행
le = LabelEncoder()
le.fit(pets['크기'])
le.transform(pets['크기'])
pets['크기_라벨링'] = le.transform(pets['크기'])

In [46]:
# 잘 변환 되었는지 확인해주기 
pets['크기_라벨링'].unique()

array([0, 2, 1])

In [47]:
# 0 : 대 / 1 : 소 / 2 : 중
pets['크기_라벨링']

0       0
1       0
2       0
3       0
4       0
       ..
7818    1
7819    1
7820    1
7822    1
7824    2
Name: 크기_라벨링, Length: 5674, dtype: int32

In [48]:
print(pets[pets['크기_라벨링'].astype(str).str.contains('0')]['크기_라벨링'].count())
print(pets[pets['크기_라벨링'].astype(str).str.contains('1')]['크기_라벨링'].count())
print(pets[pets['크기_라벨링'].astype(str).str.contains('2')]['크기_라벨링'].count())

416
4124
1134


In [49]:
# 잘 안됐네 ^^
pets['크기_라벨링'] = df['크기(라벨링)']
pets

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상,음수량,성별_라벨링,중성화_라벨링,크기,크기_라벨링
0,골든 리트리버,13,0.5,27.0,M,N,갈색,1755.0,1,0,대,2
1,골든 리트리버,10,0.5,25.0,F,N,갈색,1625.0,0,0,대,2
2,골든 리트리버,9,0.5,40.0,M,N,갈색,2600.0,1,0,대,2
3,골든 리트리버,8,0.5,28.0,F,N,갈색,1820.0,0,0,대,2
4,골든 리트리버,8,0.5,33.0,M,N,갈색,2145.0,1,0,대,2
...,...,...,...,...,...,...,...,...,...,...,...,...
7818,프렌치불독,1,0.8,4.0,M,N,흑,260.0,1,0,소,0
7819,프렌치불독,0,0.5,3.5,M,N,검정,227.5,1,0,소,0
7820,프렌치불독,0,0.5,5.2,M,N,"백색, 갈색, 흑색",338.0,1,0,소,0
7822,화이트테리어,13,0.5,7.3,F,Y,흰,474.5,0,1,소,0


# 6 클렌징한 데이터 저장 

In [50]:
pets.to_csv('pets_cleansed.csv', index = False, encoding = 'utf-8')

In [51]:
pd.read_csv('pets_cleansed.csv', encoding = 'utf-8')

,품종,나이,산책시간(시간),체중,성별,중성화여부,색상,음수량,성별_라벨링,중성화_라벨링,크기,크기_라벨링
0,골든 리트리버,13,0.5,27.0,M,N,갈색,1755.0,1,0,대,2
1,골든 리트리버,10,0.5,25.0,F,N,갈색,1625.0,0,0,대,2
2,골든 리트리버,9,0.5,40.0,M,N,갈색,2600.0,1,0,대,2
3,골든 리트리버,8,0.5,28.0,F,N,갈색,1820.0,0,0,대,2
4,골든 리트리버,8,0.5,33.0,M,N,갈색,2145.0,1,0,대,2
...,...,...,...,...,...,...,...,...,...,...,...,...
5669,프렌치불독,1,0.8,4.0,M,N,흑,260.0,1,0,소,0
5670,프렌치불독,0,0.5,3.5,M,N,검정,227.5,1,0,소,0
5671,프렌치불독,0,0.5,5.2,M,N,"백색, 갈색, 흑색",338.0,1,0,소,0
5672,화이트테리어,13,0.5,7.3,F,Y,흰,474.5,0,1,소,0
